# 20 years of LEO
In this example we will consider the tracked population of objects orbiting in Low Earth Orbit and evolve it for twenty years while counting collisions and removing decayed objects.
We start, as always, with some imports:

In [1]:
# core imports
import pykep as pk
import numpy as np
import pickle as pkl

import cascade as csc
from copy import deepcopy
from tqdm.notebook import tqdm
import heyoka as hy

from mpl_toolkits.mplot3d import Axes3D
from matplotlib import pyplot as plt
%matplotlib inline

## Loading the initial LEO population
To define the initial LEO population as tracked by the [US Space Surveillance Network (SSN)](https://en.wikipedia.org/wiki/United_States_Space_Surveillance_Network). The necessary steps are described by the code described in [The current LEO population](<./../utilities/leo_population.ipynb>)

The file needed is:
* **leo_population.pk** - created by [The current LEO population](<./../utilities/leo_population.ipynb>) notebook.

In [2]:
# r is in km and v in km/s
with open("data/leo_population.pk", "rb") as file:
    r_ic,v_ic,to_satcat_index,satcat = pkl.load(file)

* **r_ic**: contains the initial position of all satellites to be simulated (in km)
* **v_ic**: contains the initial velocity of all satellites to be simulated (in km/sec)
* **to_satcat_index**: contains the indexes in the satcat of the corresponding r_ic,v_ic entry
* **satcat**: the database created of all tracked objects

The object described by the entry ```satcat[to_satcat_index[j]]``` has initial position ```r_ic[j]``` and initial velocity ```v_ic[j]```. Let us inspect one entry

In [3]:
print("Dictionary entry: ", satcat[to_satcat_index[3685]])
print("Position (km): ", r_ic[3685])
print("Velocity (km/s): ", v_ic[3685])

Dictionary entry:  {'CCSDS_OMM_VERS': '2.0', 'COMMENT': 'GENERATED VIA SPACE-TRACK.ORG API', 'CREATION_DATE': '2022-02-03T04:23:25', 'ORIGINATOR': '18 SPCS', 'OBJECT_NAME': 'SL-14 DEB', 'OBJECT_ID': '1978-100F', 'CENTER_NAME': 'EARTH', 'REF_FRAME': 'TEME', 'TIME_SYSTEM': 'UTC', 'MEAN_ELEMENT_THEORY': 'SGP4', 'EPOCH': '2022-02-02T22:42:05.237280', 'MEAN_MOTION': '12.56769207', 'ECCENTRICITY': '0.00286110', 'INCLINATION': '82.3526', 'RA_OF_ASC_NODE': '268.9057', 'ARG_OF_PERICENTER': '297.9015', 'MEAN_ANOMALY': '75.7071', 'EPHEMERIS_TYPE': '0', 'CLASSIFICATION_TYPE': 'U', 'NORAD_CAT_ID': '19133', 'ELEMENT_SET_NO': '999', 'REV_AT_EPOCH': '93528', 'BSTAR': '0.02113600000000', 'MEAN_MOTION_DOT': '0.00003527', 'MEAN_MOTION_DDOT': '0.0000000000000', 'SEMIMAJOR_AXIS': '7814.445', 'PERIOD': '114.580', 'APOAPSIS': '1458.668', 'PERIAPSIS': '1413.952', 'OBJECT_TYPE': 'DEBRIS', 'RCS_SIZE': 'MEDIUM', 'COUNTRY_CODE': 'CIS', 'LAUNCH_DATE': '1978-10-26', 'SITE': 'PKMTR', 'DECAY_DATE': None, 'FILE': '329

on top of all the info distributed from the [US Space Surveillance Network (SSN)](https://en.wikipedia.org/wiki/United_States_Space_Surveillance_Network) we have added to the satcat (see [The current LEO population](<./../utilities/leo_population.ipynb>) notebook)an estimate of the object radius which we will need to define the various collisional radii. As we will be using a dynamics that also models the atmospheric drag via the BSTAR coefficient, we create the array to be used as parameter when instantiating a :class:`~cascade.sim`.

In [4]:
# Array containing the BSTAR coefficient in the SI units used
BSTARS = []
RADIUS = []
for idx in to_satcat_index:
    BSTARS.append(float(satcat[idx]["BSTAR"]))
    RADIUS.append(float(satcat[idx]["RADIUS"]))
# We transform the BSTAR in SI units
BSTARS = np.array(BSTARS) / pk.EARTH_RADIUS
RADIUS = np.array(RADIUS)
# .. and remove negative BSTARS (this can happen for objects that where performing orbital manouvres during the tle definition) setting the value to zero in those occasions.
BSTARS[BSTARS<0] = 0.
# We also transform r_ic and v_ic in SI
r_ic = r_ic*1000
v_ic = v_ic*1000

# Building the dynamical system to integrate
The dynamics in the LEO environment is dominated by drag and gravitational effects. The effect of the Moon gravity, Sun gravity and solar radiation pressure are not considered, even if they would not add much complexity in this simulation. We make use of :class:`~cascade.dynamics.simple_earth` defining the analytical expressions for such a dynamics.

In [5]:
dyn =  csc.dynamics.simple_earth(J2=True, C22S22=True,sun=False,moon=False,SRP=False,drag=True)

and we inspect it visually to check that all is defined as expected (the analytical expression are long and complex, but this will not be an issue for heyoka).

In [6]:
dyn

[(x, vx),
 (y, vy),
 (z, vz),
 (vx,
  ((((((-398600440779972.44 * x) * pow((x**2 + y**2 + z**2), -1.5000000000000000)) + (((-1.7555131752869966e+19 / (2.0000000000000000 * sqrt((x**2 + y**2 + z**2)))) * x) * ((3.0000000000000000 / (x**2 + y**2 + z**2)**2) - ((15.000000000000000 * z**2) / ((x**2 + y**2 + z**2)**2 * (x**2 + y**2 + z**2)))))) + ((((((7.6591108648176013e+17 / (2.0000000000000000 * pow((x**2 + y**2 + z**2), 3.5000000000000000))) * ((x * cos((4.8949608921188084 + (7.2921158548340406e-05 * t)))) + (y * sin((4.8949608921188084 + (7.2921158548340406e-05 * t)))))) * (((-x * sin((4.8949608921188084 + (7.2921158548340406e-05 * t)))) + (y * cos((4.8949608921188084 + (7.2921158548340406e-05 * t)))))**2 - ((x * cos((4.8949608921188084 + (7.2921158548340406e-05 * t)))) + (y * sin((4.8949608921188084 + (7.2921158548340406e-05 * t)))))**2)) + ((1.5318221729635203e+17 / pow((x**2 + y**2 + z**2), 2.5000000000000000)) * ((x * cos((4.8949608921188084 + (7.2921158548340406e-05 * t)))) + (y *

# We setup the simulation

the global cascade logger is here informed of the level of information we want to be reported to screen during the simulation.

In [11]:
csc.set_logger_level_info()

We now define the radius that will be used to check for decayed objects. We will assume that once the position of some object is below 150km altitude, the object can be considered as decayed.

In [12]:
reentry_radius = pk.EARTH_RADIUS+150000.

In [13]:
def remove_particle(idx, r_ic, v_ic, BSTARS,to_satcat, RADIUS):
    r_ic = np.delete(r_ic, idx, axis=0)
    BSTARS = np.delete(BSTARS, idx, axis=0)
    v_ic = np.delete(v_ic, idx, axis=0)
    to_satcat = np.delete(to_satcat, idx, axis=0)
    RADIUS = np.delete(RADIUS, idx, axis=0)
    return r_ic, v_ic, BSTARS, to_satcat, RADIUS

inside_the_radius = np.where(np.linalg.norm(r_ic,axis=1) < reentry_radius)[0]
print("Removing ", len(inside_the_radius), " orbiting objects:")
for idx in inside_the_radius:
    print(satcat[to_satcat_index[idx]]["OBJECT_NAME"], "-", satcat[to_satcat_index[idx]]["OBJECT_ID"])
r_ic, v_ic, BSTARS,to_satcat, RADIUS = remove_particle(inside_the_radius, r_ic, v_ic, BSTARS,to_satcat_index, RADIUS)

Removing  0  orbiting objects:


We can now instantiate the cascade simulation. This will trigger the JIT compilation of the numerical integrators with the selected dynamics with event detection and thus take a few seconds. 
Note that this is to be paid only once and as far as the dynamics is kept the same simulations can be made reusing the same simulation object.

As a collisional timestep, a parameter that can be tuned to get the best efficiency, we use the arbitrary value of the ISS orbital period divided by 25. 

In [14]:
# Prepare the data in the shape expected by the simulation object.
ic_state = np.hstack([r_ic, v_ic, RADIUS.reshape((r_ic.shape[0], 1))])
BSTARS = BSTARS.reshape((r_ic.shape[0], 1))
# The collisional timestep is set to 1/25 of the ISS orbital period
collisional_step = 90*60 / 25
sim = csc.sim(ic_state, collisional_step, dyn=dyn, pars=BSTARS, reentry_radius=reentry_radius)

# We run the simulation

In [15]:
final_t = 365.25 * pk.DAY2SEC * 20
print("Starting the simulation:", flush=True)

current_year = 0
while sim.time < final_t:
    years_elapsed = sim.time * pk.SEC2DAY // 365.25

    if years_elapsed == current_year:
        with open("out/year_"+str(current_year)+".pk", "wb") as file:
            pkl.dump((sim.state, sim.pars, to_satcat_index), file)
        current_year += 1

    oc = sim.step()

    if oc == csc.outcome.collision:
        pi, pj = sim.interrupt_info
        # We log the event to file
        satcat_idx1 = to_satcat_index[pi]
        satcat_idx2 = to_satcat_index[pj]
        days_elapsed = sim.time * pk.SEC2DAY
        with open("out/collision_log.txt", "a") as file_object:
            file_object.write(
                f"{days_elapsed}, {satcat_idx1}, {satcat_idx2}, {sim.state[pi]}, {sim.state[pj]}\n")
        # We log the event to screen
        o1, o2 = satcat[satcat_idx1]["OBJECT_TYPE"], satcat[satcat_idx2]["OBJECT_TYPE"]
        s1, s2 = satcat[satcat_idx1]["RCS_SIZE"], satcat[satcat_idx2]["RCS_SIZE"]
        print(
            f"\nCollision detected, {o1} ({s1}) and {o2} ({s2}) after {days_elapsed} days\n")
        # We remove the objects and restart the simulation
        sim.remove_particles([pi,pj])
        to_satcat_index = np.delete(to_satcat_index, [max(pi,pj)], axis=0)        
        to_satcat_index = np.delete(to_satcat_index, [min(pi,pj)], axis=0)


    elif oc == csc.outcome.reentry:
        pi = sim.interrupt_info
        # We log the event to file
        satcat_idx = to_satcat_index[pi]
        days_elapsed = sim.time * pk.SEC2DAY
        with open("out/decay_log.txt", "a") as file_object:
            file_object.write(f"{days_elapsed},{satcat_idx}\n")
        # We log the event to screen
        print(satcat[satcat_idx]["OBJECT_NAME"].strip(
        ) + ", " + satcat[satcat_idx]["OBJECT_ID"].strip() + ", ", days_elapsed, "REMOVED")
        # We remove the re-entered object and restart the simulation
        sim.remove_particles([pi])
        to_satcat_index = np.delete(to_satcat_index, [pi], axis=0)      

Starting the simulation:
COSMOS 1408 DEB, 1982-092EZ,  0.0011340601727223452 REMOVED
COSMOS 1408 DEB, 1982-092FZ,  0.004104128502777978 REMOVED
SL-4 R/B, 2006-061B,  0.00572006651910536 REMOVED
FREGAT DEB, 2011-037EL,  0.007307253737132345 REMOVED
STARLINK-2023, 2021-009BK,  0.018195437495537425 REMOVED
COSMOS 1408 DEB, 1982-092TR,  0.026522936514469367 REMOVED
COSMOS 1408 DEB, 1982-092ADK,  0.028751478490792105 REMOVED
COSMOS 1408 DEB, 1982-092AGV,  0.053821930334685485 REMOVED
OBJECT J, 2021-002J,  0.06110911532366596 REMOVED
RESURS O1 DEB, 1994-074BA,  0.07330302827907693 REMOVED
FREGAT DEB, 2011-037NP,  0.07449452724976004 REMOVED
STARLINK-1906, 2020-074AC,  0.2142516031021901 REMOVED


KeyboardInterrupt: 